# 자연어 처리
>   사람이 사용하는 언어의 의미를 분석하여 컴퓨터가 처리할 수 있도록 하는 과정입니다. <br />
주로 스팸처리, 맞춤법 검사, 단어 검색, 객체 인식, 질의 응답, 글 요약, 유사 단어 바꿔쓰기, 챗봇 등에 사용됩니다.

## 자연어 처리 관련 용어

*   말뭉치(corpus) : 모델을 학습시키기 위한 데이터이며, 특정한 목적으로 표본을 추출한 집합입니다.
*   토큰 : 자연어 처리를 위해 문서를 작은 단위로 나누는데, 이때 문서를 나누는 단위가 토큰입니다.
*   불용어 : 문장 내에서 많이 등장하는 단어입니다. 분석과 관계없는 단어이므로 사전에 제거합니다.
*   어간 추출 : 단어를 기본 형태로 만드는 작업입니다. ex) buys, bought => buy
*   품사 태킹 : 주어진 문장에서 품사를 식별하기 위해 붙여 주는 태그를 의미합니다.

## 전처리
*   머신러닝이나 딥러닝에서는 텍스트 자체를 특성으로 사용할 수 없으므로 변경이 필요하다.
*   자연어 처리에서 전처리는 토큰화, 불용어 제거, 어간 추출 작업등을 수행한다.

### (1) 결측치 확인
*   NAN으로 표시된 데이터 값이 결측값이다.

In [1]:
import pandas as pd
df = pd.read_csv(r'C:\Users\rladl\Downloads\딥러닝텐서플로교과서_예제파일\chap9\data\class2.csv')
df

,Unnamed: 0,id,tissue,class,class2,x,y,r
0,0,mdb000,C,CIRC,N,535.0,475.0,192.0
1,1,mdb001,A,CIRA,N,433.0,268.0,58.0
2,2,mdb002,A,CIRA,I,NaN,NaN,NaN
3,3,mdb003,C,CIRC,B,NaN,NaN,NaN
4,4,mdb004,F,CIRF,I,488.0,145.0,29.0
5,5,mdb005,F,CIRF,B,544.0,178.0,26.0


In [2]:
df.isnull().sum()

Unnamed: 0    0
id            0
tissue        0
class         0
class2        0
x             2
y             2
r             2
dtype: int64

In [3]:
df.isnull().mean()

Unnamed: 0    0.000000
id            0.000000
tissue        0.000000
class         0.000000
class2        0.000000
x             0.333333
y             0.333333
r             0.333333
dtype: float64

In [4]:
df.isnull().sum() / len(df)

Unnamed: 0    0.000000
id            0.000000
tissue        0.000000
class         0.000000
class2        0.000000
x             0.333333
y             0.333333
r             0.333333
dtype: float64

### 결측치 처리하기

In [5]:
# how='all' 모든 행이 NAN인 경우 삭제
df = df.dropna(how='all')
df

,Unnamed: 0,id,tissue,class,class2,x,y,r
0,0,mdb000,C,CIRC,N,535.0,475.0,192.0
1,1,mdb001,A,CIRA,N,433.0,268.0,58.0
2,2,mdb002,A,CIRA,I,NaN,NaN,NaN
3,3,mdb003,C,CIRC,B,NaN,NaN,NaN
4,4,mdb004,F,CIRF,I,488.0,145.0,29.0
5,5,mdb005,F,CIRF,B,544.0,178.0,26.0


In [6]:
df1 = df.dropna()
df1

,Unnamed: 0,id,tissue,class,class2,x,y,r
0,0,mdb000,C,CIRC,N,535.0,475.0,192.0
1,1,mdb001,A,CIRA,N,433.0,268.0,58.0
4,4,mdb004,F,CIRF,I,488.0,145.0,29.0
5,5,mdb005,F,CIRF,B,544.0,178.0,26.0


In [7]:
# 결측값 0으로 채우기
df2 = df.fillna(0)
df2

,Unnamed: 0,id,tissue,class,class2,x,y,r
0,0,mdb000,C,CIRC,N,535.0,475.0,192.0
1,1,mdb001,A,CIRA,N,433.0,268.0,58.0
2,2,mdb002,A,CIRA,I,0.0,0.0,0.0
3,3,mdb003,C,CIRC,B,0.0,0.0,0.0
4,4,mdb004,F,CIRF,I,488.0,145.0,29.0
5,5,mdb005,F,CIRF,B,544.0,178.0,26.0


In [9]:
# 칼럼이 x인 열의 결측값을 x의 평균값으로 바꾸기
df['x'].fillna(df['x'].mean(), inplace=True)
df

,Unnamed: 0,id,tissue,class,class2,x,y,r
0,0,mdb000,C,CIRC,N,535.0,475.0,192.0
1,1,mdb001,A,CIRA,N,433.0,268.0,58.0
2,2,mdb002,A,CIRA,I,500.0,NaN,NaN
3,3,mdb003,C,CIRC,B,500.0,NaN,NaN
4,4,mdb004,F,CIRF,I,488.0,145.0,29.0
5,5,mdb005,F,CIRF,B,544.0,178.0,26.0


### 토큰화
*   주어진 텍스트를 단어 / 문자 단위로 자르는 것을 의미한다. 
*   문장 토큰화, 단어 토큰화 2가지가 존재한다.

>   문장 토큰화 : 마침표(.), 느낌표(!), 물음표(?) 등 문장의 마지막을 뜻하는 기호에 따라 분리한다. <br />
>   단어 토큰화 : 띄어쓰기를 기준으로 문장을 구분한다.


In [10]:
# 문장 토큰화
from nltk import sent_tokenize
text_sample = 'Natural Language Processing, or NLP, is the process of extracting the meaning, or intent, behind human language. In the field of Conversational artificial intelligence (AI), NLP allows machines and applications to understand the intent of human language inputs, and then generate appropriate responses, resulting in a natural conversation flow.'
tokenized_sentences = sent_tokenize(text_sample)
print(tokenized_sentences)

['Natural Language Processing, or NLP, is the process of extracting the meaning, or intent, behind human language.', 'In the field of Conversational artificial intelligence (AI), NLP allows machines and applications to understand the intent of human language inputs, and then generate appropriate responses, resulting in a natural conversation flow.']


In [11]:
# 단어 토큰화
from nltk import word_tokenize
sentence = "This book is for deep learning learners"
words = word_tokenize(sentence)
print(words)

['This', 'book', 'is', 'for', 'deep', 'learning', 'learners']


In [12]:
# 케라스를 이용한 단어 토큰화
from tensorflow.keras.preprocessing.text import text_to_word_sequence
sentence = "it’s nothing that you don’t already know except most people aren’t aware of how their inner world works."
words = text_to_word_sequence(sentence)
print(words)

['it’s', 'nothing', 'that', 'you', 'don’t', 'already', 'know', 'except', 'most', 'people', 'aren’t', 'aware', 'of', 'how', 'their', 'inner', 'world', 'works']


## 한글 토큰화 ( KoNLPy)

In [14]:
import csv
from konlpy.tag import Okt
from gensim.models import word2vec

# 파일 불러오기
f = open(r'C:\Users\rladl\Downloads\딥러닝텐서플로교과서_예제파일\chap9\data\ratings_train.txt', 'r', encoding='utf-8')
rdr = csv.reader(f, delimiter='\t')
rdw = list(rdr)
f.close()

In [15]:
# 한글 분석기 호출
twitter = Okt()

result = []
for line in rdw :
    # 형태소 분석 / line[1] 텍스트 부분 선택
    malist = twitter.pos(line[1], norm=True, stem=True) # norm : 품사 추출 / stem : 어근 추출 
    r = []
    for word in malist :  # 분석된 형태소 하나씩 word에 입력됨
        if not word[1] in ["Josa", "Eomi", "Punctuation"]: 
            r.append(word[0])
    rl = (" ".join(r)).strip()
    result.append(rl)
    print(rl)

document
아 더빙 진짜 짜증나다 목소리
흠 포스터 보고 초딩 영화 줄 오버 연기 가볍다 않다
너 무재 밓었 다그 래서 보다 추천 다
교도소 이야기 구먼 솔직하다 재미 없다 평점 조정
사이 몬페 그 의 익살스럽다 연기 돋보이다 영화 스파이더맨 늙다 보이다 하다 커스틴 던스트 너무나도 이쁘다 보이다
막 걸음 마 떼다 3 세 초등학교 1 학년 생인 8 살다 영화 ㅋㅋㅋ 별 반개 아깝다 움
원작 긴장감 제대로 살리다 하다
별 반개 아깝다 욕 나오다 이응경 길용우 연 기 생활 몇 년 정말 발 해도 그것 낫다 납치 감금 반복 반복 이 드라마 가족 없다 연기 못 하다 사람 모 엿 네
액션 없다 재미 있다 몇 안되다 영화
왜 이렇게 평점 낮다 꽤 볼 한 데 헐리우드 식 화려하다 너무 길들이다 있다
걍 인피니트 짱 진짜 짱 ♥
볼때 눈물나다 죽다 90년 대의 향수 자극 허진호 감성 절제 멜로 달인
울면 손 들 횡단보도 건너다 때 뛰다 치다 올 뻔 이범수 연기 드럽다 못 하다
담백하다 깔끔하다 좋다 신 문 기 사 로만 보다 보다 자꾸 잊어버리다 그 들 사람 이다 것
취향 존중 한 다지 진짜 내생 극장 보다 영화 중 가장 노잼 노 감동 임 스토리 어거지 감동 어거지
ㄱ 냥 매번 긴장 되다 재밌다 ㅠㅠ
차다 사람 들 웃기다 바스코 이기 락스 코 끄다 바비 이기 아이돌 깔다 그냥 끄다 안달 것 보이다
굿바이 레닌 표절 것 이해 하다 왜 뒤 갈수록 재미 없어지다
이 것 정말 깨알 캐스팅 질퍽 하 않다 산뜻하다 내 용구성 자다 버무러진 깨알 일드 ♥
약탈 자 위 변명 이르다 저 놈 들 착하다 놈 들 절대 아니다 걸
나름 심오하다 뜻 있다 듯 그냥 학생 선생 놀다 영화 절대 아니다
보다 웃다 않다 건 불가능하다
재미없다 지루하다 같다 음식 영화 도 바베트 만찬 넘다 차이나다 바베트 만찬 이야기 있다 음식 보다 재미 있다 이 것 볼 없다 음식 별로 안 나오다 핀란드 풍경 구 경 할랫 늘다 그것 별로 안 나오다 ㅡㅡ
절대 평범하다 영화 아니다 수작 는걸 말씀드리다
주제 좋다 중반 지루

KeyboardInterrupt: 

In [21]:
cnt = 0
for line in rdw:
    if cnt == 3 :
        break
    print(line)
    cnt += 1
    
print("==================================")

cnt = 0
for line in rdw :
    if cnt == 3:
        break
    # 형태소 분석
    malist = twitter.pos(line[1], norm=True, stem=True) # line[1] 텍스트 부분 선택
    r = []
    for word in malist : # 분석된 형태소 하나씩 word에 입력됨
        if not word[1] in ["Josa", "Eomi", "Punctuation"]:
            r.append(word[0])
    rl = (" ".join(r)).strip()
    print(rl)
    cnt += 1

['id', 'document', 'label']
['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0']
['3819312', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1']
document
아 더빙 진짜 짜증나다 목소리
흠 포스터 보고 초딩 영화 줄 오버 연기 가볍다 않다


In [28]:
mylist = '나는 데이터 사이언스 입니다.'

malist = twitter.pos(mylist, norm=True, stem=True)
for word in malist:
    print(word,'\n')

('나', 'Noun') 

('는', 'Josa') 

('데이터', 'Noun') 

('사이언스', 'Noun') 

('이다', 'Adjective') 

('.', 'Punctuation') 



### 불용어 제거
*   문장 내에서 빈번하게 발생하여 의미를 부여하기 어려운 단어들을 의미합니다. ex( a, the, 는, 를, 을)

In [31]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize

sample_text = "One of the first things that we ask ourselves is what are the pros and cons of any task we perform."
text_tokens = word_tokenize(sample_text)
tokens_without_sw = [word for word in text_tokens if not word in stopwords.words('english')]

print("불용어 제거 미적용:", text_tokens, '\n')
print("불용어 제거 적용:",tokens_without_sw)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rladl\AppData\Roaming\nltk_data...


불용어 제거 미적용: ['One', 'of', 'the', 'first', 'things', 'that', 'we', 'ask', 'ourselves', 'is', 'what', 'are', 'the', 'pros', 'and', 'cons', 'of', 'any', 'task', 'we', 'perform', '.'] 

불용어 제거 적용: ['One', 'first', 'things', 'ask', 'pros', 'cons', 'task', 'perform', '.']


[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rladl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 어간 추출
*   단어 원형을 찾아 주는 것을 의미합니다.
*   어간 추출은 단어 그 자체만 고려하기 때문에 품사가 달라도 사용 가능합니다.

In [32]:
# 어근 추출
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

print(stemmer.stem('obesses'),stemmer.stem('obssesed'))
print(stemmer.stem('standardizes'),stemmer.stem('standardization'))
print(stemmer.stem('national'), stemmer.stem('nation'))
print(stemmer.stem('absentness'), stemmer.stem('absently'))
print(stemmer.stem('tribalical'), stemmer.stem('tribalicalized'))

obess obsses
standard standard
nation nation
absent absent
tribal tribalic


In [33]:
#랭커스터 (lancaster) 알고리즘
from nltk.stem import LancasterStemmer
stemmer = LancasterStemmer()

print(stemmer.stem('obsesses'),stemmer.stem('obsessed'))
print(stemmer.stem('standardizes'),stemmer.stem('standardization'))
print(stemmer.stem('national'), stemmer.stem('nation'))
print(stemmer.stem('absentness'), stemmer.stem('absently'))
print(stemmer.stem('tribalical'), stemmer.stem('tribalicalized'))

obsess obsess
standard standard
nat nat
abs abs
trib trib


### 표제어 추출
*   단어 원형을 찾아주는 것을 의미합니다.
*   표제어 추출은 단어가 문장 속에서 어떤 품사로 쓰였는지 고려하기 때문에 품사가 같아야 사용이 가능합니다.
*   품사와 같은 문법뿐만 아니라 문장내에서 단어 의미도 고려하기 때문에 성능이 좋습니다. (하지만 시간이 오래 걸리는 단점)

In [36]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [37]:
#표제어 추출(Lemmatization)
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemma = WordNetLemmatizer()

print(stemmer.stem('obsesses'),stemmer.stem('obsessed'))
print(lemma.lemmatize('standardizes'),lemma.lemmatize('standardization'))
print(lemma.lemmatize('national'), lemma.lemmatize('nation'))
print(lemma.lemmatize('absentness'), lemma.lemmatize('absently'))
print(lemma.lemmatize('tribalical'), lemma.lemmatize('tribalicalized'))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rladl\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


obsess obsess
standardizes standardization
national nation
absentness absently
tribalical tribalicalized


### 정규화
*   자연어 처리에서는 표현 방법이 다른 단어들을 통합시켜서 같은 단어로 만들어 줍니다.

In [38]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.python.data import Dataset
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models
from tensorflow.keras import layers

### 정규화하지 않은 모델

In [42]:
df = pd.read_csv(r'C:\Users\rladl\Downloads\딥러닝텐서플로교과서_예제파일\chap9\data\covtype.csv')
x = df[df.columns[:54]]
y = df.Cover_Type

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=90)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu',
                         input_shape=(x_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(8, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

history1 = model.fit(x_train, y_train, epochs=26, batch_size=60, validation_data=(x_test, y_test))

Epoch 1/26
6779/6779 [==============================] - 32s 5ms/step - loss: 3.1111 - accuracy: 0.5648 - val_loss: 1.4102 - val_accuracy: 0.6005
Epoch 2/26
6779/6779 [==============================] - 31s 5ms/step - loss: 1.0792 - accuracy: 0.6414 - val_loss: 0.9848 - val_accuracy: 0.6531
Epoch 3/26
6779/6779 [==============================] - 28s 4ms/step - loss: 0.7359 - accuracy: 0.6945 - val_loss: 0.6810 - val_accuracy: 0.7196
Epoch 4/26
6779/6779 [==============================] - 34s 5ms/step - loss: 0.6466 - accuracy: 0.7209 - val_loss: 0.6433 - val_accuracy: 0.7335
Epoch 5/26
6779/6779 [==============================] - 27s 4ms/step - loss: 0.6135 - accuracy: 0.7349 - val_loss: 0.6469 - val_accuracy: 0.7051
Epoch 6/26
6779/6779 [==============================] - 29s 4ms/step - loss: 0.5914 - accuracy: 0.7437 - val_loss: 0.6083 - val_accuracy: 0.7346
Epoch 7/26
6779/6779 [==============================] - 30s 4ms/step - loss: 0.5748 - accuracy: 0.7526 - val_loss: 0.5452 - val_ac

---

### 정규화한 모델

In [43]:
from sklearn import preprocessing
df = pd.read_csv(r'C:\Users\rladl\Downloads\딥러닝텐서플로교과서_예제파일\chap9\data\covtype.csv')
x = df[df.columns[:55]]
y = df.Cover_Type
x_train, x_test, y_train, y_test = train_test_split(x, y , train_size = 0.7, random_state =  90)

# 정규화가 필요한 열 선택
train_norm = x_train[x_train.columns[0:10]]
test_norm = x_test[x_test.columns[0:10]]

# 정규화 ( x_train에만 fit하고 x_test는 transform()만 한다. )
std_scale = preprocessing.StandardScaler().fit(train_norm)
x_train_norm = std_scale.transform(train_norm)
training_norm_col = pd.DataFrame(x_train_norm, index=train_norm.index, columns=train_norm.columns) 
x_train.update(training_norm_col)
print (x_train.head())

x_test_norm = std_scale.transform(test_norm)
testing_norm_col = pd.DataFrame(x_test_norm, index=test_norm.index, columns=test_norm.columns) 
x_test.update(testing_norm_col)
print (x_test.head())

        Elevation    Aspect     Slope  Horizontal_Distance_To_Hydrology  \
152044   0.222366 -0.228639 -0.412503                          0.148486   
363373   1.980490 -0.469989  0.255453                          3.018822   
372733  -1.081933  0.271939  0.389044                         -0.867895   
572846  -1.164122 -0.157128 -0.278912                         -1.267860   
114145  -0.052787  0.861906  0.255453                         -0.279711   

        Vertical_Distance_To_Hydrology  Horizontal_Distance_To_Roadways  \
152044                        0.149095                         1.336119   
363373                        4.443372                         0.168073   
372733                       -0.160093                        -0.241801   
572846                       -0.795646                        -0.461170   
114145                       -0.125739                         1.811419   

        Hillshade_9am  Hillshade_Noon  Hillshade_3pm  \
152044       1.002687        0.539776     

In [44]:
model = tf.keras.Sequential([
         tf.keras.layers.Dense(64, activation='relu',                  
         input_shape=(x_train.shape[1],)),
         tf.keras.layers.Dense(64, activation='relu'),
         tf.keras.layers.Dense(8, activation=  'softmax')
 ])

model.compile(optimizer= tf.keras.optimizers.Adam(0.001),
                                loss='sparse_categorical_crossentropy',
                                metrics=['accuracy'])
history2 = model.fit(
                 x_train, y_train,
                 epochs= 26, batch_size = 60,
                 validation_data = (x_test, y_test))

Epoch 1/26
6779/6779 [==============================] - 32s 4ms/step - loss: 0.0324 - accuracy: 0.9910 - val_loss: 7.6192e-05 - val_accuracy: 1.0000
Epoch 2/26
6779/6779 [==============================] - 35s 5ms/step - loss: 2.7569e-04 - accuracy: 0.9999 - val_loss: 7.8946e-06 - val_accuracy: 1.0000
Epoch 3/26
6779/6779 [==============================] - 27s 4ms/step - loss: 2.4868e-04 - accuracy: 0.9999 - val_loss: 3.9568e-05 - val_accuracy: 1.0000
Epoch 4/26
6779/6779 [==============================] - 25s 4ms/step - loss: 2.4674e-04 - accuracy: 0.9999 - val_loss: 6.0254e-06 - val_accuracy: 1.0000
Epoch 5/26
6779/6779 [==============================] - 28s 4ms/step - loss: 1.5084e-04 - accuracy: 1.0000 - val_loss: 2.5848e-05 - val_accuracy: 1.0000
Epoch 6/26
6779/6779 [==============================] - 28s 4ms/step - loss: 1.0379e-04 - accuracy: 1.0000 - val_loss: 4.4081e-04 - val_accuracy: 0.9999
Epoch 7/26
6779/6779 [==============================] - 36s 5ms/step - loss: 1.6541e-0

---

>   정규화 하지 않은 모델은 val_accuracy : 0.79, val_loss : 0.49 이며 <br />
정규화한 모델은 val_accuracy : 1.0, val_loss : 8.9959e-08

정규화한 모델이 훨씬 정확도도 높으며, 오차도 훨씬 작은 것을 알 수 있다. <br />
모델 학습 전에 데이터의 범위가 다르면 정규화하는 것이 좋음을 알 수 있다.